# 1. Introducción

El siguiente cuaderno realiza la conversion de valores de temperatura en Celsius a Fahrenheit o viceversa. Se pretende simular un gran conjunto de lecturas que se almacenan en un vector. Se simulan las lecturas con valores seudoaleatorios a los cuales les podemos definir un techo y piso por parametros como asi la cantidad de estos. Las formulas de conversión son las siguientes:

De Celsius a Farenheit:  <center> $F$°= ( $C$° * 9/5 ) + 32 </center>

De Farenheit a Celsius:  <center> $C$°= ( $F$° + 32 ) * 5/9 </center>

El objetivo es realizar la conversion de manera más rápida utilizando el paralelismo que nos da CUDA, en este caso trabajando con una sola dimensión.

# 2. Armado del ambiente

*   Se instala el módulo de CUDA en el cuaderno.
*  Se importan las bibliotecas Necesarias


In [ ]:
!pip install pycuda

from datetime import datetime
import random
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# 3. Desarrollo




## 3.1 Ejecucion Secuencial

In [ ]:
# --------------------------------------------
#@title 3.1.1 Parámetros de ejecución { vertical-output: true }

cantidad_lecturas =   500000#@param {type: "number"}
conversor = "Grado Fahrenheit a Grado Celsius" #@param ["Grado Fahrenheit a Grado Celsius", "Grado Celsius a Grado Fahrenheit"]
temp_min =  -100 #@param {type:"slider", min:-100, max:0, step:1}
temp_max = 1000  #@param {type:"slider", min:0, max:1000, step:10}
# --------------------------------------------

tiempo_total_secuencial = datetime.now()

#Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#Simplifico la direccion de conversion

if(conversor=="Grado Celsius a Grado Fahrenheit"):
  direccion_conversion=1
else:
  direccion_conversion=0

# Genero la lista con las lecturas
temperaturas = []
for i in range(0,cantidad_lecturas):
  temperaturas+=[round(random.uniform(temp_min, temp_max),2)]

#Realizo el calculo

tiempo_cpu = datetime.now()

temperaturas_resultado=[]

for i in range(0,cantidad_lecturas-1):
  if(direccion_conversion==1):
    temperaturas_resultado+= [(temperaturas[i] * 9/5)+32]
  else:
    temperaturas_resultado+= [(temperaturas[i] + 32)*5/9]

tiempo_cpu = datetime.now() - tiempo_cpu
tiempo_total_secuencial = datetime.now() - tiempo_total_secuencial
print( "Cantidad de elementos: ", cantidad_lecturas )
print("Tiempo Total: ", tiempo_en_ms( tiempo_total_secuencial ), "[ms]" )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_cpu   ), "[ms]" )
    

Cantidad de elementos:  500000
Tiempo Total:  590.145 [ms]
Tiempo CPU:  140.297 [ms]


##3.2 Ejecución paralela con CUDA

In [ ]:
# --------------------------------------------
#@title 3.2.1 Parámetros de ejecución { vertical-output: true }

cantidad_lecturas =   500000#@param {type: "number"}
conversor = "Grado Fahrenheit a Grado Celsius" #@param ["Grado Fahrenheit a Grado Celsius", "Grado Celsius a Grado Fahrenheit"]
temp_min =  -100 #@param {type:"slider", min:-100, max:0, step:1}
temp_max = 1000  #@param {type:"slider", min:0, max:1000, step:10}
# --------------------------------------------

#CPU - Tomo el tiempo inicial total
tiempo_total_cuda = datetime.now()

#Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#CPU - Simplifico la direccion de conversion

if(conversor=="Grado Celsius a Grado Fahrenheit"):
  direccion_conversion=1
else:
  direccion_conversion=0

# CPU - Genero las lecturas
temperaturas = []
for i in range(0,cantidad_lecturas):
  temperaturas+=[round(random.uniform(temp_min, temp_max),2)]

# CPU - Defino la memoria de los vectores en cpu.
temperaturas_cpu = numpy.asarray(temperaturas)
temperaturas_cpu = temperaturas_cpu.astype( numpy.float32() )
temperaturas_resultado_cpu = numpy.empty_like(temperaturas_cpu)

# GPU - reservo la memoria GPU.
temperaturas_gpu = cuda.mem_alloc(temperaturas_cpu.nbytes)
temperaturas_resultado_gpu = cuda.mem_alloc( temperaturas_resultado_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( temperaturas_gpu, temperaturas_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void conversor_temperatura( int n, int direccion_conversion, float *temp_original, float *temp_resultado )
{
  #define CONST1 9/5
  #define CONST2 5/9
  #define CONST3 32

  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    if(direccion_conversion==1)
    { 
      temp_resultado[idx] = (temp_original[idx] *CONST1)+CONST3;
    }else{
      temp_resultado[idx] = (temp_original[idx]-CONST3)*CONST2;
    }
   
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("conversor_temperatura")

#CPU - Tomo el tiempo inicial de ejecución de la función kernel.
tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_lecturas+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

kernel( numpy.int32(cantidad_lecturas),numpy.int32(direccion_conversion), temperaturas_gpu, temperaturas_resultado_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

#CPU - Tomo el tiempo final de ejecución de la función kernel.
tiempo_gpu = datetime.now() - tiempo_gpu

# CPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( temperaturas_resultado_cpu, temperaturas_resultado_gpu )

#CPU- Tomo el tiempo total final.
tiempo_total_cuda = datetime.now() - tiempo_total_cuda

#CPU - Informo los resultados.
print( "Cantidad de elementos: ", cantidad_lecturas )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo Total: ", tiempo_en_ms( tiempo_total_cuda ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )


Thread x:  256 , Bloque x: 1954
Cantidad de elementos:  500000
Thread x:  256 , Bloque x: 1954
Tiempo Total:  479.194 [ms]
Tiempo GPU:  1.146 [ms]


# 4. Tabla de pasos

> Se define la tabla de pasos del punto 3.2




Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de paramtros desde Colab.
CPU      |  direccion_conversion= | Simplifico la direccion de conversion
CPU      | temperaturas+=[round(random.uniform(temp_min, temp_max),2)] | Genero las lecturas
CPU      | numpy.asarray() | Defino la memoria de los vectores en cpu
GPU      | cuda.mem_alloc()| Reservo la memoria GPU
GPU      | cuda.memcpy_htod() | Copio la memoria al GPU
CPU      | SourceModule()| Defino la función kernel que ejecutará en GPU
CPU      | module.get_function()| Genero la función kernel
CPU      | dim_tx/dim_bx | Calcula las dimensiones.
GPU      | kernel()| Ejecuta el kernel
CPU      | cuda.memcpy_dtoh()| Copio el resultado desde la memoria GPU
CPU      | print()| Informo los resultados.

# 5. Conclusiones

Habiendo ejecutado varias veces el ejercicio con una cantidad de elementos igual a 500000 se puede apreciar que cuando no se utiliza CUDA, se tarda un 30% mas en ejecutar. Lo interesante es que si tomamos solo el tiempo en que se ejecuta la funcion paralela podemos apreciar que se ejecuta mas de 350 veces mas rápido. Esto sucede debido a la preparación por el lado del CPU que conlleva ejecutar estas tareas en CUDA desde pyton, ya sea preparar datos para que sean compatibles como generar el codigo fuente o pasar informacion entre ambas memorias.

Definitivamente para grandes volumenes de datos la ejecución paralela hace lo imposible posible pero si nuestra cantidad de datos no es lo suficientemente grande, todo lo que conlleva ejecutar en CUDA no vale la pena.

Para esta conclusión se ejecutaron 10 veces los ejercicios teniendo como resultados promedios:

CPU - 3.1:

*   Tiempo Total:  597.601 [ms]
*   Tiempo CPU:  134.583 [ms]

GPU - 3.2:

*   Tiempo Total:  456.59 [ms]
*   Tiempo GPU:  0.382 [ms]






#6. Bibliografía

Se utilizaron las bibliografías otorgadas por la catedra junto con:

* https://www.digikey.com/es/resources/conversion-calculators/conversion-calculator-temperature
*  https://j2logo.com/python/generar-numeros-aleatorios-en-python/
*  https://www.mclibre.org/consultar/python/lecciones/python-listas.html

